# (PR4-3) Datawarehouse

The tasks

In [19]:
#Bouw voort op je uitwerking van het vorige practicum:

#1 Maak een database in SSMS, dit wordt je uiteindelijke Data Warehouse.
#Helemaal goed.

#2 Maak een ETL-schema van elke tabel in het Sterschema.
#Dit kan in het document worden gevonden.

#3 Implementeer elk gemaakt ETL-schema in Python (met Jupyter Notebook). Gebruik hierbij de technische informatie uit week 2.
#Helemaal goed.

#4 Zorg er ten slotte voor dat je het Sterschema en elk ETL-schema aan je portfolio hebt toegevoegd. Hier word je in week 6 op beoordeeld.
#Dit kan in het document worden gevonden.

## DWH imports

Import the necassery modules.

In [20]:
import pyodbc as sql_server
from dotenv import dotenv_values as values
import pandas as pd
import sqlite3 as sql
import warnings as warn
warn.simplefilter("ignore")

## Connection to SQL Server

Load the values from the .env.local file.

In [21]:
env: dict[str, str | None] = values(".env.local")

conn_str: any = (
    "DRIVER={SQL Server};Server=" + env["SERVER"] + 
    ";Database=" + env["DATABASE"] + 
    ";User ID=" + env["USERNAME"] + 
    ";Password=" + env["PASSWORD"] + 
    "trusted_connection=yes;"
)

try:
    conn = sql_server.connect(conn_str)
    print(f"Succesfully established connection to the server: {conn}")

except sql_server.Error as err:
    print(f"Error while connecting to the server: {err.args[1]}")
    raise

Succesfully established connection to the server: <pyodbc.Connection object at 0x000001B376F42660>


## Read the GO Databases/CSV's

Read the original datasources.

In [22]:
conn_db1 = sql.connect("Great_Outdoors_Data_SQLite/go_crm.sqlite")
conn_db2 = sql.connect("Great_Outdoors_Data_SQLite/go_sales.sqlite")
conn_db3 = sql.connect("Great_Outdoors_Data_SQLite/go_staff.sqlite")

go_csv: list[str] = ["GO_SALES_INVENTORY_LEVELSData", "GO_SALES_PRODUCT_FORECASTData"]
conn_csv = [pd.read_csv("Great_Outdoors_Data_SQLite/" + csv + ".csv") for csv in go_csv]

## Read the original GO Databases

Read the original db sources

In [23]:
cursor1 = conn_db1.cursor()
cursor2 = conn_db2.cursor()
cursor3 = conn_db3.cursor()

cursors = [cursor1, cursor2, cursor3]
conns = [conn_db1, conn_db2, conn_db3]

for cursor, _ in zip(cursors, conns):
    datas = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

    for data in datas:
        dfs = cursor.execute(f"SELECT * FROM {data[0]}").fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame(dfs, columns=columns)
        df = df[df.columns.drop(list(df.filter(regex="TRIAL")))]
        print(df)

    cursor.close()

  AGE_GROUP_CODE UPPER_AGE LOWER_AGE
0              1        20         0
1              2        30        21
2              3        40        31
3              4        50        41
4              5        60        51
5              6        70        61
   COUNTRY_CODE      COUNTRY_EN FLAG_IMAGE SALES_TERRITORY_CODE
0             1          France        F01                    6
1            10          Sweden        F10                    5
2            11           Japan        F11                    4
3            12          Taiwan        F12                    4
4            13           Korea        F13                    4
5            14           China        F14                    4
6            16       Australia        F15                    4
7            17         Belgium        F16                    6
8            18         Denmark        F17                    5
9            19           Spain        F18                    7
10            2         Germany      

## Insert the GO Databases/CSV's to the datewarehouse

Original datasources, will be inserted into the datawarehouse.

In [24]:
#Insertion


## Read all tables

Read the tables with pandas from SQL Server.

In [25]:
product = pd.read_sql_query("SELECT * FROM product", conn)
sales_product_forecast = pd.read_sql_query("SELECT * FROM sales_product_forecast", conn)
sales_inventory_levels = pd.read_sql_query("SELECT * FROM sales_inventory_levels", conn)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", conn)
sales_targetdata = pd.read_sql_query("SELECT * FROM sales_targetdata", conn)
course = pd.read_sql_query("SELECT * FROM course", conn)
training = pd.read_sql_query("SELECT * FROM training", conn)

satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type", conn)
satisfaction = pd.read_sql_query("SELECT * FROM returned_reason", conn)
returned_reason = pd.read_sql_query("SELECT * FROM returned_reason", conn)
returned_item = pd.read_sql_query("SELECT * FROM returned_item", conn)
retailer_contact = pd.read_sql_query("SELECT * FROM retailer_contact", conn)
order_method = pd.read_sql_query("SELECT * FROM order_method", conn)
# order = pd.read_sql_query("SELECT * FROM order_", conn)

## Read the new datawarehouse database

Get the values of the datewarehouse

In [26]:
#DWH data
df = retailer_contact
df

,sales_territory_id,sales_territory_name_en,country_id,country_en,country_flag_image,country_sales_territory_code,retailer_site_code,retailer_site_address1,retailer_site_address2,retailer_site_city,...,retailer_contact_jobposition_en,retailer_contact_extension,retailer_contact_gender,retailer_contact_age_group_code,retailer_contact_age_group_upper,retailer_contact_age_group_lower,retailer_contact_sales_demographic_code,retailer_contact_sales_demographic_codemr,retailer_contact_sales_demographic_age_group_code,retailer_contact_sales_demographic_sales_percent


In [27]:
import copy
import datetime

Creating a slowly changing dimension (assuming type 2) for every dimension in the datawarehouse

In [28]:
# Product
## Copy
product_scd = copy.deepcopy(product)

## Add columns
product_scd["Nummer_sk"] = range(len(product_scd))
product_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
product_scd.set_index("Nummer_sk", inplace=True)

# Sales_product_forecast
## Copy
sales_product_forecast_scd = copy.deepcopy(sales_product_forecast)

## Add columns
sales_product_forecast_scd["Nummer_sk"] = range(len(product_scd))
sales_product_forecast_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
sales_product_forecast_scd.set_index("Nummer_sk", inplace=True)

# Sales_inventory_levels
## Copy
sales_inventory_levels_scd = copy.deepcopy(sales_inventory_levels)

## Add columns
sales_inventory_levels_scd["Nummer_sk"] = range(len(sales_inventory_levels_scd))
sales_inventory_levels_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
sales_inventory_levels_scd.set_index("Nummer_sk", inplace=True)

# Sales_staff
## Copy
sales_staff_scd = copy.deepcopy(sales_staff)

## Add columns
sales_staff_scd["Nummer_sk"] = range(len(sales_staff_scd))
sales_staff_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
sales_staff_scd.set_index("Nummer_sk", inplace=True)

# Sales_targetdata
## Copy
sales_targetdata_scd = copy.deepcopy(sales_targetdata)

## Add columns
sales_targetdata_scd["Nummer_sk"] = range(len(sales_targetdata_scd))
sales_targetdata_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
sales_targetdata_scd.set_index("Nummer_sk", inplace=True)

# Course
## Copy
course_scd = copy.deepcopy(course)

## Add columns
course_scd["Nummer_sk"] = range(len(course_scd))
course_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
course_scd.set_index("Nummer_sk", inplace=True)

# Training
## Copy
training_scd = copy.deepcopy(training)

## Add columns
training_scd["Nummer_sk"] = range(len(training_scd))
training_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
training_scd.set_index("Nummer_sk", inplace=True)

# Satisfaction_type
## Copy
satisfaction_type_scd = copy.deepcopy(satisfaction_type)

## Add columns
satisfaction_type_scd["Nummer_sk"] = range(len(satisfaction_type_scd))
satisfaction_type_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
satisfaction_type_scd.set_index("Nummer_sk", inplace=True)

# Satisfaction
## Copy
satisfaction_scd = copy.deepcopy(satisfaction)

## Add columns
satisfaction_scd["Nummer_sk"] = range(len(satisfaction_scd))
satisfaction_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
satisfaction_scd.set_index("Nummer_sk", inplace=True)

# Returned_reason
## Copy
returned_reason_scd = copy.deepcopy(returned_reason)

## Add columns
returned_reason_scd["Nummer_sk"] = range(len(returned_reason_scd))
returned_reason_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
returned_reason_scd.set_index("Nummer_sk", inplace=True)

# Returned_item
## Copy
returned_item_scd = copy.deepcopy(returned_item)

## Add columns
returned_item_scd["Nummer_sk"] = range(len(returned_item_scd))
returned_item_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
returned_item_scd.set_index("Nummer_sk", inplace=True)

# Retailer_contact
## Copy
retailer_contact_scd = copy.deepcopy(retailer_contact)

## Add columns
retailer_contact_scd["Nummer_sk"] = range(len(retailer_contact_scd))
retailer_contact_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
retailer_contact_scd.set_index("Nummer_sk", inplace=True)

# Order_method
## Copy
order_method_scd = copy.deepcopy(order_method)

## Add columns
order_method_scd["Nummer_sk"] = range(len(order_method_scd))
order_method_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
order_method_scd.set_index("Nummer_sk", inplace=True)

# Order
## Copy
# order_scd = copy.deepcopy(order)

## Add columns
# order_scd["Nummer_sk"] = range(len(order_scd))
# order_scd["Timestamp"] = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M")

## Fix index
# order_scd.set_index("Nummer_sk", inplace=True)

Testing the relations are still valid

In [29]:
# TODO: perform testing